# Social Readers

Here's a Twitter reader for most recent 500 tweets containing keyword/s, also included is a new function for getting all posts and comments in a given subreddit. Note that running the twitter side of this will require API keys which I can provide.

In [2]:
#Import Various Libraries, including Tweepy, a Python library for the Twitter API.
import os
import requests
import pandas as pd
from dotenv import load_dotenv
from textblob import TextBlob #Old sentiment analysis library, presumably not as good as vader's.
#%matplotlib inline
import tweepy
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

load_dotenv()

True

In [3]:
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\mnewk\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [4]:
analyzer = SentimentIntensityAnalyzer()

In [5]:
consumer_key = os.getenv('TWITTER_API_KEY')
consumer_secret = os.getenv('TWITTER_SECRET_KEY')
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(auth)

In [6]:
dict_tweet_structure = {"twitter_user":"","category":[],"time":"","compound_sentiment":"","text":"","tweet_id":"",
                        "tweet_source":"","pos_sentiment":0,"neu_sentiment":0,"neg_sentiment": 0,"retweet_count":0,"fav_count":0,
                         "Popularity Rating":""}

In [7]:
df_tweets_found = pd.DataFrame(dict_tweet_structure)
#df_tweets_found.to_csv('tweets.csv')

In [8]:
def tweet_search_free(string_query, number_tweets = 100):
    for tweets in tweepy.Cursor(api.search, q=string_query).items(number_tweets):
        screen_name = tweets.user.screen_name
        contents = tweets.text
        #polarity = sentiment_reader(contents)
        compound, pos, neu, neg = sentiment_reader(contents)
        category = 'bitcoin'
        yield [screen_name, category, str(tweets.created_at), compound, contents, tweets.id, tweets.source,pos,
               neu, neg, tweets.retweet_count, tweets.favorite_count, (compound * int(tweets.retweet_count))]

In [9]:
def sentiment_reader(text_block):
    #sentiment_text = TextBlob(text_block) ###Old method using TextBlob
    #return sentiment_text.polarity ###Old method using TextBlob
    text_sentiment = analyzer.polarity_scores(text_block)
    sentiment =  text_sentiment["compound"]
    pos = text_sentiment["pos"]
    neu = text_sentiment["neu"]
    neg = text_sentiment["neg"]
    return sentiment, pos, neu, neg

In [10]:
def tweet_dataframe_append(rows, target_dataframe):
    for row in rows:
        series_result = pd.Series(row, index=target_dataframe.columns)
        target_dataframe = target_dataframe.append(series_result, ignore_index=True)
        return target_dataframe

In [11]:
#Returns df of all posts/comments in a subreddit before & after given time
def get_redd(subreddit, after, before): .
    df_redd = pd.DataFrame()
    while before > after:
        url = f"https://api.pushshift.io/reddit/search/submission/?subreddit={subreddit}&sort=desc&sort_type=created_utc&after={after}&before={before}&size=1000"
        request = requests.get(url)
        json_response = request.json()
        df_response = pd.json_normalize(json_response['data'])
        df_redd = pd.concat([df_redd,df_response])
        before = json_response['data'][-1]['created_utc']
    return df_redd

SyntaxError: invalid syntax (<ipython-input-11-f9c91ab7ef5e>, line 2)

In [ ]:
df_tweets_live = pd.DataFrame(dict_tweet_structure)
query = 'bitcoin OR btc'
print(query)
live_results = tweet_search_free(query, 1000)
for tweets in live_results:
    #print (tweets)
    df_tweets_live = tweet_dataframe_append(live_results, df_tweets_live)

In [53]:
df_tweets_live

,twitter_user,category,time,compound_sentiment,text,tweet_id,tweet_source,pos_sentiment,neu_sentiment,neg_sentiment,retweet_count,fav_count,Popularity Rating
0,ColinMc99043404,bitcoin,2021-03-13 20:07:28,0.6369,BITCOIN Strategy Finally Revealed By Michael S...,1370828880957743105,Twitter Web App,0.259,0.741,0.0,0,0,0.0000
1,xgil4x4,bitcoin,2021-03-13 20:07:28,0.0000,RT @frankothafifth: My boxers watching me put ...,1370828878927835140,Twitter for iPhone,0.000,1.000,0.0,634,0,0.0000
2,byhawk01,bitcoin,2021-03-13 20:07:27,0.0000,"RT @MzlmYaar: #bitcoin bu kadar yükselmişken, ...",1370828877027803138,Twitter for Android,0.000,1.000,0.0,3,0,0.0000
3,thorswhu,bitcoin,2021-03-13 20:07:27,0.4588,@NickGreenr Prizes in $BTC could be lit AF,1370828876193103872,Twitter for iPad,0.300,0.700,0.0,0,0,0.0000
4,ashavhicriego77,bitcoin,2021-03-13 20:07:27,0.0000,RT @ScottZILWarrior: I will send one random pe...,1370828875522015234,Twitter for iPhone,0.000,1.000,0.0,1694,0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Ayodelebrain,bitcoin,2021-03-13 20:04:39,0.6848,RT @Wealth_Theory: #Bitcoin is making me wealt...,1370828172900569095,Twitter for Android,0.204,0.796,0.0,29,0,19.8592
496,johnmorganFL,bitcoin,2021-03-13 20:04:39,0.0000,"XRP, Bitcoin SV, Elrond Price Analysis: 13 Mar...",1370828171113865220,Twitter Web App,0.000,1.000,0.0,0,1,0.0000
497,ATL_ATH,bitcoin,2021-03-13 20:04:39,0.0000,🚀 ATH breached for USD-BTC on Bittrex!\nNew A...,1370828169742315529,ATL_ATH,0.000,1.000,0.0,0,0,0.0000
498,YoCrypto,bitcoin,2021-03-13 20:04:38,0.7832,📌 #BTC $BTC update\n\nThat’s how you capture a...,1370828169163407362,Twitter for iPhone,0.276,0.724,0.0,0,0,0.0000


In [16]:
import nest_asyncio
nest_asyncio.apply()
import twint
import time
from datetime import datetime

In [ ]:
five_ago = round(time.time(),0) - 300
c = twint.Config()
c.Store_object = True
c.Pandas =True
c.Search = "bitcoin OR btc"
c.Limit = 500
c.Lang = 'en'
c.Since = datetime.utcfromtimestamp(five_ago).strftime('%Y-%m-%d %H:%M:%S')
fd = twint.run.Search(c)

CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 1.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 8.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 27.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 64.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 125.0 secs


In [ ]:
!pip3 uninstall twint

In [ ]:
!pip3 install --user --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint